In [ ]:
# 載入模組
from urllib.request import Request, urlopen
from datetime import datetime
from IPython.display import HTML, display
import time
import json
import pandas as pd
import base64

# {'全名': ['代號', ([資料開始日]，[資料結束日])]}, ex: 'Bitcoin': {['BTC', ([2013, 5, 5], [2021, 2, 27])]}
_search = {填入虛幣}

def download_csv(df, title='Download CSV file', filename='data.csv'):
  csv = df.to_csv(index=False)
  payload = base64.b64encode(csv.encode()).decode()
  a = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
  html = a.format(payload=payload,title=title,filename=filename)
  return display(HTML(html))

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'}

url_id = '填入ID的URL'
r_id = Request(url_id, headers=headers)
data_id = json.loads(urlopen(r_id).read())
 
list_id = []
columns_id = []
info_id = []

# 撈出所有幣
for j in range(len(data_id['data']['cryptoCurrencyMap'])):
  values = []
  for k, v in data_id['data']['cryptoCurrencyMap'][j].items():
    if j == 1:
      columns_id.append(k)
    values.append(v)
    
    if k == 'symbol':
      break
  info_id.append(values)
 
df_id = pd.DataFrame.from_dict(info_id)
df_id.columns = columns_id
# print(f'{df_id}\n')

columns_historical = []
columns_overview = ['Date', 'Price']

for k1, v1 in _search.items():
  print(f'{k1}:')
  _id = str(df_id[(df_id['name'] == k1) & (df_id['symbol'] == v1[0])]['id'].iloc[0])
  time_start = str(int(time.mktime(datetime(v1[1][0][0], v1[1][0][1], v1[1][0][2]).timetuple())))
  time_end = str(int(time.mktime(datetime(v1[1][1][0], v1[1][1][1], v1[1][1][2]).timetuple())))
  list_id.append([_id, time_start, time_end])

  url_historical = f'填入Historical的URL'
  r_historical = Request(url_historical, headers=headers)
  data_historical = json.loads(urlopen(r_historical).read())

  info_historical = []

  if not columns_historical:
    _columns = []

  for j in range(len(data_historical['data']['quotes'])):
    values = []
    for k2, v2 in data_historical['data']['quotes'][j]['quote']['USD'].items():
      if j == 1:
        if not columns_historical:
          _columns.append(k2)
      if k2 == 'timestamp':
        v2 = datetime.strptime(v2, '%Y-%m-%dT%H:%M:%S.%fz').strftime('%Y-%m-%d')
      values.append(v2)
    info_historical.append(values)

  if not columns_historical:
    columns_historical = _columns

  df_historical = pd.DataFrame.from_dict(info_historical)
  df_historical.columns = columns_historical
  print(f'{df_historical}\n')
  download_csv(df_historical, title=f'Download {_search[k1][0]} Historical CSV file', filename=f'{_search[k1][0]}_historical.csv')
  print('\n')

  url_overview = f'填入Overview的URL'
  r_overview = Request(url_overview, headers=headers)
  data_overview = json.loads(urlopen(r_overview).read())
 
  info_overview = []
 
  for k3, v3 in data_overview['data'].items():
    info_overview.append([datetime.strptime(k3, '%Y-%m-%dT%H:%M:%S.%fz').strftime('%Y-%m-%d'), data_overview['data'][k3]['USD'][0]])
 
  df_overview = pd.DataFrame.from_dict(info_overview)
  df_overview.columns = columns_overview
  print(f'{df_overview}\n')
  download_csv(df_overview, title=f'Download {_search[k1][0]} Overview CSV file', filename=f'{_search[k1][0]}_overview.csv')
  print('\n')
